In [ ]:
import numpy as np
import torch
from torch import nn
from torch.nn.functional import pad
from torchvision import models
from transformers import AutoTokenizer, AutoModel


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tae898/emoberta-base')
model = AutoModel.from_pretrained('tae898/emoberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/407 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at tae898/emoberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
tokenizer

RobertaTokenizerFast(name_or_path='tae898/emoberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [41]:
import pandas as pd

data = pd.read_csv('dev_sent_emo.csv')
data.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"


In [ ]:
speaker2id = { sp:i for i, sp in enumerate(data['Speaker'].unique()) }

data['Speaker_ID'] = data['Speaker'].apply(lambda x: speaker2id[x])

data.head()

In [ ]:
class M2FNetTextEmbedder:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('tae898/emoberta-base')
        self.encoder = AutoModel.from_pretrained('tae898/emoberta-base')
        self.model_max_tokens = self.tokenizer.model_max_length

    def encode(self, diag_id):
        diag_uttrs = data[data['Dialogue_ID'] == diag_id]

        max_tokens = self.model_max_tokens - 2
        samples_ids = []
        attn_masks = []

        for i, row in diag_uttrs.iterrows():
            sequence = []
            target_uttr = tokenizer(self.tokenizer.eos_token*2 + \
                             f"{row['Speaker'].upper()}:{row['Utterance']}" + \
                                self.tokenizer.eos_token*2, add_special_tokens=False)
            sequence.append(target_uttr.input_ids)
            seq_len = len(target_uttr.input_ids)
            shift = 1
            while seq_len <= max_tokens:
                prev_idx = row['Utterance_ID'] - shift
                next_idx = row['Utterance_ID'] + shift
                if prev_idx >= 0:
                    uttr_row = diag_uttrs[diag_uttrs['Utterance_ID'] == prev_idx].iloc[0]
                    uttr_str = f"{uttr_row['Speaker'].upper()}:{uttr_row['Utterance']} "
                    uttr_tokens = tokenizer(uttr_str, add_special_tokens=False).input_ids

                    sequence.insert(0, uttr_tokens)
                if next_idx < len(diag_uttrs['Utterance_ID']):
                    uttr_row = diag_uttrs[diag_uttrs['Utterance_ID'] == next_idx].iloc[0]
                    uttr_str = f"{uttr_row['Speaker'].upper()}:{uttr_row['Utterance']} "
                    uttr_tokens = tokenizer(uttr_str, add_special_tokens=False).input_ids

                    sequence.append(uttr_tokens)
                if prev_idx < 0 and next_idx >= diag_uttrs['Utterance_ID'].max():
                    break
                shift += 1
                seq_len = sum([len(ut) for ut in sequence])
            cls_tok = tokenizer(tokenizer.cls_token, add_special_tokens=False).input_ids
            eos_tok = tokenizer(tokenizer.eos_token, add_special_tokens=False).input_ids
            if seq_len > max_tokens:
                sequence = sequence[1:-1]
            sequence = [cls_tok] + sequence + [eos_tok]
            seq_flatten = torch.tensor([tok for uttr in sequence for tok in uttr], dtype=torch.long)
            input_ids = pad(seq_flatten, (0, 512-len(seq_flatten)), value=1).unsqueeze(0)
            attention_mask = torch.ones(512)
            attention_mask[(512-len(seq_flatten)):] = 0
            samples_ids.append(input_ids)
            attn_masks.append(attention_mask.unsqueeze(0))

        samples_ids_tensor = torch.stack(samples_ids).squeeze(1)
        attn_masks_tensor = torch.stack(attn_masks).squeeze(1)
        embs = self.encoder(input_ids=samples_ids_tensor, attention_mask=attn_masks_tensor).last_hidden_state[:, 0, :]
        
        return embs

In [ ]:
class SpeakerEmb1TextEmbedder:
    def __init__(self, n_speakers, speaker_lambda):
        self.m2fnet_embedder = M2FNetTextEmbedder()
        self.speaker_emb = nn.Embedding(n_speakers, 768)
        self.gru = nn.GRU(768, 768 // 2, bidirectional=True, num_layers=1)
        self.speaker_lambda = speaker_lambda
    
    def encode(self, diag_id):
        diag_frame = data[data['Dialogue_ID'] == diag_id]
        diag_embs = self.m2fnet_embedder.encode(diag_id) # (uttr_count, hid_dim)
        diag_features, _ = self.gru(diag_embs)
        speakers = torch.tensor(np.array(diag_frame['Speaker_ID'], dtype=np.int32), dtype=torch.long)
        speaker_embs = self.speaker_emb(speakers) # (uttr_count, hid_dim)
        text_features = diag_features + speaker_embs

        return text_features

In [ ]:
embeddings = []
# embedder = M2FNetTextEmbedder()
embedder = SpeakerEmb1TextEmbedder(len(speaker2id))

Some weights of RobertaModel were not initialized from the model checkpoint at tae898/emoberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for diag_id in [0, 1]: # data['Dialogue_ID'].unique()
    embs = embedder.encode(diag_id)
    if embeddings is None:
        embeddings = embs
    else:
        embeddings = torch.cat([embeddings, embs], dim=0)